# Introduction

In this notebook, we experiment with the optimal histogram algorithm. We will implement a simple version based on recursion and you will do the hard job of implementing a dynamic programming-based version. 


References: 
* H. V. Jagadish, Nick Koudas, S. Muthukrishnan, Viswanath Poosala, Kenneth C. Sevcik, Torsten Suel: Optimal Histograms with Quality Guarantees. VLDB 1998: 275-286. (url: http://engineering.nyu.edu/~suel/papers/vopt.pdf)
* Dynamic Programming (wikipedia): https://en.wikipedia.org/wiki/Dynamic_programming




1. V-optimal histograms are an example of a more "exotic" histogram. V-optimality is a Partition Rule which states that the bucket boundaries are to be placed as to minimize the cumulative weighted variance of the buckets. Implementation of this rule is a complex problem and construction of these histograms is also a complex process. （大意：最优直方图就是我们定义的一个桶内方差最小，不过大概不能改变数据的排列顺序）

2. 大致的思想，使用动态规划，全局最优必然是局部最优，因而我们可以一层层拆解，取每一次拆解的最优的方式。拆解过程又是一个递归的过程。

In [1]:
LARGE_NUM = 1000000000.0
EMPTY = -1

# DEBUG = 2
# DEBUG = 1
DEBUG = 0

import numpy as np

# 计算划分好坏程度的函数
def sse(arr):
    if len(arr) == 0: # deal with arr == []
        return 0.0

    avg = np.average(arr)
    val = sum( [(x-avg)*(x-avg) for x in arr] )
    return val

# 为了打印方便的函数
def calc_depth(b):
    return 5 - b

# 核心部分
def v_opt_rec(xx, b):
    mincost = LARGE_NUM
    n = len(xx)

    # check boundary condition:
    if n < b: # 没法划分
        return LARGE_NUM + 1
    elif b == 1: # 最简单的情况
        return sse(xx)
    else:  # the general case
        if DEBUG > 1:
            #print('.. BEGIN: input = {!s:<30}, b = {}'.format(xx, b))
            print('..{}BEGIN: input = {!s:<30}, b = {}'.format('  '*calc_depth(b), xx, b))

            
        for t in range(n):
            prefix = xx[0 : t+1]
            suffix = xx[t+1 : ]
            cost = sse(prefix) + v_opt_rec(suffix, b - 1)
            mincost = min(mincost, cost)

        if DEBUG > 0:
            #print('.. END: input = {!s:<32}, b = {}, mincost = {}'.format(xx, b, mincost))
            print('..{}END: input = {!s:<32}, b = {}, mincost = {}'.format('  '*calc_depth(b), xx, b, mincost))

        return mincost



Now, try to understand how the algorithm works -- feel free to modify the code to output more if you need. Specifically, 

1. Observe and understand how the recursion works (set `DEBUG = 2`)
2. Observe and understand how many sub-problems are being solved again and again (set `DEBUG = 1`), especially when the input array is longer. 

In [2]:
x = [7, 9, 13, 5]
b = 3

c = v_opt_rec(x, b)
print('optimal cost = {}'.format(c))

optimal cost = 2.0


In [3]:
x = [1, 3, 9, 13, 17]
b = 4

c = v_opt_rec(x, b)
print('c = {}'.format(c))

c = 2.0


In [4]:
x = [3, 1, 18, 9, 13, 34, 17]
b = 4

c = v_opt_rec(x, b)
print('c = {}'.format(c))

c = 42.66666666666667


In [5]:
x = [1, 2, 3, 4, 5, 6]
b = 4

c = v_opt_rec(x, b)
print('c = {}'.format(c))

c = 1.0


In [6]:
x = [1, 7, 8, 15, 21, 30, 78]
b = 4

c = v_opt_rec(x, 4)
print('c = {}'.format(c))

c = 46.666666666666664


## Exercise

Now you need to implement the same algorithm using dynamic programming. The idea is to fill in a table, named `opt`, of $b \times n$, where `opt[i][j]` records the optimal cost for building a histogram of $[x_j, x_{j+1}, \ldots, x_n]$ using $i$ bins. 

The first step is to work out the general recursive formula, in the form of: 
$$
opt[i][j] = \min_{t} f(t)
$$
You need to work out what is the domain of $t$ and what exactly is $f()$, which should depend on $opt[u][v]$ that has **already** been computed. If you cannot work it out directly, you can observe the sub-problems being solved in the recursive algorithm and see if you can schedule the computation of table cells such that every cell required on the right hand side (i.e., $f(t)$) is always scheduled **before** the current cell $opt[i][j]$. 

In [7]:
# 注：
#    这里没有实现，但大概的思路和上面一样，先从最简单的情况计算，然后增加选择最优的一种。

def fill_opt(xx, b):
    opt = np.zeros((b, len(xx)))
    m, n = opt.shape
    for i in range(m):
        for j in range(n):
            opt[i][j] = v_opt_rec(xx[j:], i + 1)
    return opt
    
    

In [8]:
fill_opt([1, 7, 8, 15, 21, 30, 78], 4)

array([[  4.10685714e+03,   3.54950000e+03,   3.09320000e+03,
          2.46600000e+03,   1.87800000e+03,   1.15200000e+03,
          0.00000000e+00],
       [  5.59333333e+02,   3.66800000e+02,   2.61000000e+02,
          1.14000000e+02,   4.05000000e+01,   0.00000000e+00,
          1.00000000e+09],
       [  1.39250000e+02,   7.85000000e+01,   6.50000000e+01,
          1.80000000e+01,   0.00000000e+00,   1.00000000e+09,
          1.00000000e+09],
       [  4.66666667e+01,   1.85000000e+01,   1.80000000e+01,
          0.00000000e+00,   1.00000000e+09,   1.00000000e+09,
          1.00000000e+09]])

In [9]:
a = np.zeros((10, 10))
a.shape

(10, 10)